# Final Project - Evaporating Liquidity

In this project, we replicate tables from the paper "_Evaporating Liquidity_" by Stefan Nagel using the Principals of Reproducible Analytical Pipelines (RAPs) learned in the class. 

Our replication is automated from end-to-end using Pydoit, formatted using the project template (blank_project) provided by professor Bejarano, which is based on the Cookiecutter Data Science template.

In [1]:
import pandas as pd

import config

import load_CRSP_stock
import load_FF_industry
import load_vix

import clean_CRSP_stock
import calc_reversal_strategy
# import hedged_reversal_strategy

DATA_DIR= config.DATA_DIR

import warnings
warnings.filterwarnings('ignore')


## Data Collection

### 1. Pull and load CRSP data from WRDS

Using `load_CRSP_stock`, we pull and save CRSP daily stock data and indexes from WRDS (Wharton Research Data Services). 

The CRSP daily stock data is needed to construct individual portfolios based on Reversal strategy. The CRSP daily index data is needed to evaluate the performance of Reversal strategy portfolios.

Specifically:
- we use query to pull data of stocks with share code 10 or 11, from NYSE, AMEX, and Nasdaq
- pull one extra month of daily stock data for later data cleaning and processing

#### CRSP daily stock data

In [2]:
df_dsf = load_CRSP_stock.load_CRSP_daily_file(data_dir=DATA_DIR)
df_dsf.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17445472 entries, 0 to 445471
Data columns (total 12 columns):
 #   Column   Dtype         
---  ------   -----         
 0   date     datetime64[ns]
 1   permno   int64         
 2   permco   int64         
 3   exchcd   int64         
 4   prc      float64       
 5   bid      float64       
 6   ask      float64       
 7   shrout   float64       
 8   cfacpr   float64       
 9   cfacshr  float64       
 10  ret      float64       
 11  retx     float64       
dtypes: datetime64[ns](1), float64(8), int64(3)
memory usage: 1.7 GB


### CRSP daily indexes

In [3]:
df_msix = load_CRSP_stock.load_CRSP_index_files(data_dir=DATA_DIR)
df_msix.columns

Index(['caldt', 'vwretd', 'vwindd', 'vwretx', 'vwindx', 'ewretd', 'ewindd',
       'ewretx', 'ewindx', 'sprtrn', 'spindx', 'decret1', 'decind1', 'decret2',
       'decind2', 'decret3', 'decind3', 'decret4', 'decind4', 'decret5',
       'decind5', 'decret6', 'decind6', 'decret7', 'decind7', 'decret8',
       'decind8', 'decret9', 'decind9', 'decret10', 'decind10', 'totval',
       'totcnt', 'usdval', 'usdcnt'],
      dtype='object')

### 2. Pull and load data from the Fama-French Data Library



Using `load_FF_industry`, we pull and save 48 industry portfolio daily returns from the Fama/French Data Library. 

The industry portfolios are constructed by classifying stocks into 48 industries as in Fama and French (1997). The industry portfolio daily returns are needed to construct the industry portfolios based on Reversal strategy.

In [4]:
ff = load_FF_industry.load_FF_industry_portfolio_daily(data_dir=DATA_DIR)

#### Average Value Weighted Daily Returns

In [5]:
ff[0].tail()

,Agric,Food,Soda,Beer,Smoke,Toys,Fun,Books,Hshld,Clths,...,Boxes,Trans,Whlsl,Rtail,Meals,Banks,Insur,RlEst,Fin,Other
Date,,,,,,,,,,,,,,,,,,,,,
2010-12-27,0.12,-0.35,-0.22,-0.61,-0.26,0.06,-0.48,0.29,-0.55,-0.08,...,0.24,0.21,0.12,-0.13,-0.61,1.17,0.24,-0.18,0.83,0.52
2010-12-28,0.37,0.03,-0.39,0.12,-0.05,-1.45,-0.63,-0.50,0.14,-0.37,...,-0.02,-0.08,-0.16,-0.06,-0.23,0.15,-0.21,-0.39,-0.10,0.17
2010-12-29,2.22,-0.03,-0.32,0.09,0.23,0.55,0.22,0.70,-0.38,-0.53,...,0.02,0.21,0.23,0.51,0.49,-0.25,-0.09,0.70,-0.50,-0.10
2010-12-30,1.00,0.07,0.00,-0.11,-0.42,-0.02,-0.60,-0.78,-0.15,0.54,...,-0.04,-0.02,-0.03,0.13,-0.39,-0.35,-0.29,-0.47,-0.25,-0.44
2010-12-31,-0.13,0.01,-0.81,0.34,0.05,-1.12,1.14,-0.18,-0.06,-0.93,...,-1.02,0.00,-0.54,-0.48,-0.40,0.09,0.19,-0.09,0.07,0.60


### 3. Pull and load VIX from the Fama-French Data Library



Using `load_vix`, we pull and save CBOE Volatility Index data from FRED. The data is used later in table replicatation.

In [6]:
vix = load_vix.load_vix(data_dir=DATA_DIR)

## Data Cleaning and Processing

### Select the desired subsample of the data

#### For reversal strategy based on transaction prices calculated from daily closing prices

1. Adjust prc with a negative sign to be positive

> Prc is the closing price or the negative bid/ask average for a trading day. If the closing price is not available on any given trading day, the number in the price field has a negative sign to indicate that it is a bid/ask average and not an actual closing price. Please note that in this field the negative sign is a symbol and that the value of the bid/ask average is not negative. If neither closing price nor bid/ask average is available on a date, prc is set to zero. 

2. Filter stocks with closing price (prc) of at least $1 on the last trading day of the previous calendar month

> To enter into the sample, a stock needs to have share code 10 or 11. In addition, it must have a closing price of at least $1 on the last trading day of the previous calendar month.


3. Select the CRSP stock data for a specific time period

In [7]:
def select_stocks_by_closing_prices(df):
    """
    Clean the CRSP stock data for strategy based on closing prices
    """

    # make sure 'prc' is positive (negative sign means using bid/ask average instead of closing price)
    df = clean_CRSP_stock.clean_prc_to_positive(df)

    # stocks must have a closing price of at least $1 on the last trading day of the previous calendar month
    df = clean_CRSP_stock.clean_1dollar_prc(df)

    # select time range of the data
    df = clean_CRSP_stock.clean_date(df, start_year=1998, end_year=2024)

    return df.reset_index()

#### For reversal strategy based on quote-midpoints calculated from averages of closing bid and ask quotes

1. Adjust prc with a negative sign to be positive

2. Filter stocks with closing price (prc) of at least $1 on the last trading day of the previous calendar month

3. Select stocks traded on NASDAQ

4. Adjust bid and ask quotes for stock splits using CRSP adjustment factor

> Reversal strategy returns based on transaction prices are calculated from daily closing prices, and the reversal strategy returns based on quote-midpoints are calculated from averages of closing bid and ask quotes, as reported in the CRSP daily returns file (for Nasdaq stocks only), adjusted for stock splits and dividends using the CRSP adjustment factors and dividend information.


5. Filter stocks with ratio of bid to quote-midpoint not smaller than 0.5.

6. Filter stocks with one-day return based on quote-midpoints minus the return based on closing prices not less than -50% and not higher than 100%. If a closing transaction price is not available, the quote-midpoint is used to calculate transaction-price returns.

7. Select the CRSP stock data for a specific time period


> In a few instances, the closing bid and ask data for Nasdaq stocks on CRSP have some data recording errors, such as increases of bid or ask by a factor of 100 or digits that are cut off. 

> To screen out data recording errors of bid and ask data for Nasdaq stocks: require that the ratio of bid to quote-midpoint is not smaller than 0.5, and the one-day return based on quote-midpoints minus the return based on closing prices is not less than -50% and not higher than 100%. If a closing transaction price is not available, the quote-midpoint is used to calculate transaction-price returns.


In [8]:
def select_stocks_by_quote_midpoints(df):
    """
    Clean the CRSP stock data for strategy based on quote-midpoints
    """

    # make sure 'prc' is positive (negative sign means using bid/ask average instead of closing price)
    df = clean_CRSP_stock.clean_prc_to_positive(df)

    # Nasdaq stocks only
    df = df[df['exchcd'] == 3]

    # adjusting the price using adjustment factors
    df['bid'] = df['bid'] / df['cfacpr']
    df['ask'] = df['ask'] / df['cfacpr']
    df['quote_midpoint'] = (df['bid'] + df['ask']) / 2

    # ratio of bid to quote-midpoint is not smaller than 0.5
    df = clean_CRSP_stock.clean_bid_quote_midpoint(df)

    # one-day return based on quote-midpoints minus the return based on closing prices is less than -50% and higher than 100%
    df = clean_CRSP_stock.clean_one_day_return(df)

    # stocks must have a closing price of at least $1 on the last trading day of the previous calendar month
    df = clean_CRSP_stock.clean_1dollar_prc(df)

    # select time range of the data
    df = clean_CRSP_stock.clean_date(df, start_year=1998, end_year=2024)

    return df.reset_index()

### Load cleaned data

In [9]:
dfcp = clean_CRSP_stock.load_CRSP_closing_price(data_dir=DATA_DIR)
dfcp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16530172 entries, 0 to 16530171
Data columns (total 13 columns):
 #   Column   Dtype         
---  ------   -----         
 0   index    int64         
 1   date     datetime64[ns]
 2   permno   int64         
 3   permco   int64         
 4   exchcd   int64         
 5   prc      float64       
 6   bid      float64       
 7   ask      float64       
 8   shrout   float64       
 9   cfacpr   float64       
 10  cfacshr  float64       
 11  ret      float64       
 12  retx     float64       
dtypes: datetime64[ns](1), float64(8), int64(4)
memory usage: 1.6 GB


In [10]:
dfmid = clean_CRSP_stock.load_CRSP_midpoint(data_dir=DATA_DIR)
dfmid.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10250715 entries, 0 to 10250714
Data columns (total 16 columns):
 #   Column                    Dtype         
---  ------                    -----         
 0   index                     int64         
 1   date                      datetime64[ns]
 2   permno                    int64         
 3   permco                    int64         
 4   exchcd                    int64         
 5   prc                       float64       
 6   bid                       float64       
 7   ask                       float64       
 8   shrout                    float64       
 9   cfacpr                    float64       
 10  cfacshr                   float64       
 11  ret                       float64       
 12  retx                      float64       
 13  quote_midpoint            float64       
 14  transaction_price_return  float64       
 15  quote_midpoint_return     float64       
dtypes: datetime64[ns](1), float64(11), int64(4)
memory u

## Reversal Strategy 

### Construct Reversal Strategy 

Each day t, the reversal strategy returns are calculated as the average of returns from five reversal strategies that weight stocks proportional to the negative of market-adjusted returns on days t − 1, t − 2, ..., t − 5, with weights scaled to add up to $1 short and $1 long.

In [11]:
def calc_reverse_strategy_ret(df, type_col='industry', ret_col='ret'):
    df = df.copy()
    df['ret-avg'] = df.groupby('date')[ret_col].transform(lambda x: x - x.mean())
    df['w'] = df.groupby('date')['ret-avg'].transform(lambda x: - x / (0.5 * x.abs().sum()))

    for i in range(1, 6):
        df[f'w_lag_{i}'] = df.groupby(type_col)['w'].shift(i)

    df['rev_ret'] = (df['w_lag_1'] + df['w_lag_2'] + df['w_lag_3'] + df['w_lag_4'] + df['w_lag_5']) * df[ret_col] / 5

    return df.groupby('date')['rev_ret'].sum()

To be specific, the returns of the reversal strategies are calculated as an overlay of the returns of five sub-strategies: One with portfolio weights conditioned on day t − 1 returns, one conditioned on day t − 2 data, ..., one conditioned on t − 5 data. Then we calculated the simple average of these five sub-strategies’ returns as the overall reversal strategy return.

### Hedge against market factor risk

## Table Replication

We replicate tables with data with time period from January 1998 to December 2010.

### Table 1: Summary Statistics of Reversal Strategy Returns

Panel A presents statistics for the raw returns of these strategies, while Panel B shows similar statistics for hedged returns, which are obtained by eliminating conditional market factor exposure.

#### A. Raw returns

Reversal strategy returns are calculated both with transaction-price returns, quote-midpoint returns, and industry returns. In each case, the portfolio weights are calculated with the same type of return (on days t − 1 to t − 5) as the type used to calculate portfolio returns (on day t).

- Transaction-price returns are calculated from daily CRSP closing transaction prices (for NYSE, AMEX, and Nasdaq stocks). 
- Quote-midpoint returns are calculated from bid-ask midpoints of daily CRSP closing quotes (with Nasdaq stocks only). 
- The industry returns pulled from Fama and French are calculated with transaction prices. 

In [12]:
ret_raw = calc_reversal_strategy.load_reversal_return(data_dir=DATA_DIR)
ret_raw

,Transact. prices,Quote-midpoints,Industry portfolio
date,,,
1998-01-02,1.329402,0.038368,0.134118
1998-01-05,0.689190,0.375095,-0.483562
1998-01-06,0.354757,0.066624,0.114318
1998-01-07,0.391311,0.147555,0.252857
1998-01-08,0.256501,-0.008249,-0.284123
...,...,...,...
2010-12-27,-0.265861,-0.368400,0.021103
2010-12-28,0.050559,-0.104661,-0.151843
2010-12-29,0.095505,0.580082,-0.166645


In [13]:
df_stat_A = calc_reversal_strategy.summary_stats(ret_raw)
df_stat_A = df_stat_A.style.format('{:.2f}',na_rep='')
df_stat_A


,Transact. prices,Quote-midpoints,Industry portfolio
Mean return(% per day),0.31,0.19,0.02
Std.dev.(% per day),0.56,0.67,0.56
Skewness,3.00,3.54,0.77
Kurtosis,38.37,49.58,14.58
Worst day return(%),-3.84,-4.56,-3.70
Worst 3-month return(%),2.51,-2.72,-12.17
Beta,0.11,0.09,0.10
Annualized Sharpe Ratio,8.63,4.54,0.45


#### B. Hedged returns

In [14]:
ret_hedged = calc_reversal_strategy.load_reversal_return(data_dir=DATA_DIR, hedged=True)
ret_hedged

,Hedged Transact. prices,Hedged Quote-midpoints,Hedged Industry portfolio
date,,,
1998-01-02,1.293719,0.006260,0.098217
1998-01-05,0.676658,0.366557,-0.492475
1998-01-06,0.411797,0.105485,0.154885
1998-01-07,0.444951,0.195821,0.306825
1998-01-08,0.377501,0.100629,-0.162382
...,...,...,...
2010-12-27,-0.275475,-0.377051,0.011429
2010-12-28,0.046795,-0.107225,-0.154520
2010-12-29,0.082324,0.571102,-0.176019


In [15]:
df_stat_B = calc_reversal_strategy.summary_stats(ret_hedged)
df_stat_B = df_stat_B.style.format('{:.2f}',na_rep='')
df_stat_B

,Hedged Transact. prices,Hedged Quote-midpoints,Hedged Industry portfolio
Mean return(% per day),0.30,0.19,0.01
Std.dev.(% per day),0.54,0.65,0.54
Skewness,3.01,3.80,0.65
Kurtosis,38.89,55.13,12.20
Worst day return(%),-3.05,-3.96,-3.31
Worst 3-month return(%),2.07,-2.02,-9.18
Beta,0.00,0.00,0.00
Annualized Sharpe Ratio,8.90,4.58,0.38


### Table 2: Predicting Reversal Strategy Returns with VIX

## Table Reproduction

Here, we reproduce tables with updated data.

### Table 1: Summary Statistics of Reversal Strategy Returns

### Table 2: Predicting Reversal Strategy Returns with VIX

## Analysis outside of replication